In [6]:
%%bigquery --use_rest_api  data
SELECT * FROM `qwiklabs-asl-00-748dac0c969e.enron_datasets.enron_with_labels`;

Query is running:   0%|          |

Downloading:   0%|          |

In [1]:
import pandas
from transformers import pipeline
import math
from sklearn.metrics import confusion_matrix,precision_score,recall_score,roc_curve,auc,f1_score,accuracy_score
import numpy as np


2023-06-15 05:20:45.137595: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-15 05:20:51.625113: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-06-15 05:20:51.626539: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/loca

In [2]:
classifier = pipeline("zero-shot-classification", model="roberta-large-mnli", device=0, batch_size=10)

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
candidate_labels = ["finance", "apparel and fashion", "food and dining", "health and medicine", 
                    "tech and electronics", "entertainment", "home and garden", "science and environment",
                    "government and politics", "apparel", ]
                    # "career", "banking and payments", 
                    # "sports", "personal care and beauty", "travel", "real estate", 
                    # "investment", "automotive", "consumer electronics", "social", 
                    # "sports fan", "restaurants and dining", "shoes", "casual apparel", 
                    # "cooking and recipes", "education" ]

In [9]:
def get_df_with_prediction(count, df, inp_columns):
    small_df = df.take([x for x in range(0,count)])
    arr_pred_multi_class = []
    arr_pred = []
    it = 1
    for id in small_df.index:
        print(it, end = " ")
        it +=1
        sequence = " ".join([small_df[col][id] for col in inp_columns])
        pred = classifier(sequence, candidate_labels)
        arr_pred.append(pred)
        pred_multi = classifier(sequence, candidate_labels, multi_label=True)
        arr_pred_multi_class.append(pred_multi)
    small_df['pred_multi_class'] = arr_pred_multi_class
    small_df['pred'] = arr_pred
    return small_df

def metrics(topic, y_true, y_pred, y_prob=None):        
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    acc = accuracy_score(y_true, y_pred)
    metric_map = {
        'topic': topic,
        'precision': precision, 
        'recall': recall, 
        'f1_score': f1, 
        'accuracy': acc,        
        'true negative':tn, 
        'false positive': fp,
        'false negative': fn,
        'true positive': tp,

    }
    if y_prob is not None:
        fpr, tpr, thresholds = roc_curve(y_true, y_prob)
        auc_score = auc(fpr, tpr)
        metric_map['auc'] = auc_score
    return metric_map
    

def evaluate(small_df, label_col="label"):
    result = {}
    for topic in candidate_labels:
        result[topic] = {"y_true":[], "y_pred":[], "y_pred_1":[], "y_prob":[]}
    for id in small_df.index:
        labels = [x.replace("_"," ") for x in small_df[label_col][id]]
        pred = small_df["pred"][id]
        output = pred['labels']
        pred_multi_class = small_df["pred_multi_class"][id]            
        for topic in candidate_labels:
            result[topic]["y_true"].append(1 if topic in labels else 0)
            result[topic]["y_pred_1"].append(1 if topic in output[0:3] else 0)
            y_prob = pred_multi_class['scores'][pred_multi_class['labels'].index(topic)] if topic in pred_multi_class['labels'] else 0.0
            result[topic]["y_prob"].append(y_prob)
            result[topic]["y_pred"].append(y_prob>0.3)                
                
                
    eval_result = []
    eval_result_1 = []    
    for topic in result.keys():
        eval_result.append(
            metrics(
                topic,
                result[topic]["y_true"], 
                result[topic]["y_pred"], 
                result[topic]["y_prob"]
            )
        )
        eval_result_1.append(
            metrics(
                topic,
                result[topic]["y_true"], 
                result[topic]["y_pred_1"]
            )
        )
    eval_df = pandas.DataFrame(eval_result)
    eval_df1 = pandas.DataFrame(eval_result_1)    
    return (eval_df, eval_df1)

In [ ]:
df = get_df_with_prediction(10000, data, ["subject","body"])

In [42]:
df.to_pickle("enron_10000.pickle")

In [ ]:
df1 = get_df_with_prediction(100000, data, ["subject","body"])

In [43]:
df1.to_pickle("enron_100000.pickle")

In [40]:
evaluate(df, "topics")[0]

,topic,precision,recall,f1_score,accuracy,true negative,false positive,false negative,true positive,auc
0,finance,0.184434,0.596825,0.281789,0.7125,6561,2494,381,564,0.710780
1,apparel and fashion,0.007630,0.894737,0.015131,0.7787,7770,2211,2,17,0.896493
2,food and dining,0.019717,0.884615,0.038574,0.7707,7661,2287,6,46,0.924287
3,health and medicine,0.004478,0.625000,0.008893,0.7771,7761,2223,6,10,0.795266
4,tech and electronics,0.044179,0.794393,0.083703,0.6278,6108,3678,44,170,0.797329
5,entertainment,0.014411,0.944444,0.028389,0.6509,6458,3488,3,51,0.865287
6,home and garden,0.005157,0.325000,0.010152,0.7465,7452,2508,27,13,0.520660
7,science and environment,0.016998,0.587500,0.033040,0.7249,7202,2718,33,47,0.689956
8,government and politics,0.043527,0.697509,0.081940,0.5608,5412,4307,85,196,0.660323
9,apparel,0.001315,1.000000,0.002626,0.7721,7718,2279,0,3,0.969357


In [41]:
evaluate(df, "topics")[1]

,topic,precision,recall,f1_score,accuracy,true negative,false positive,false negative,true positive
0,finance,0.182346,0.664550,0.286170,0.6867,6239,2816,317,628
1,apparel and fashion,0.011019,0.421053,0.021477,0.9271,9263,718,11,8
2,food and dining,0.043897,0.788462,0.083164,0.9096,9055,893,11,41
3,health and medicine,0.021116,0.875000,0.041237,0.9349,9335,649,2,14
4,tech and electronics,0.045849,0.691589,0.085997,0.6854,6706,3080,66,148
5,entertainment,0.007670,0.962963,0.015218,0.3270,3218,6728,2,52
6,home and garden,0.002598,0.375000,0.005161,0.4217,4202,5758,25,15
7,science and environment,0.018581,0.550000,0.035948,0.7640,7596,2324,36,44
8,government and politics,0.050685,0.790036,0.095259,0.5783,5561,4158,59,222
9,apparel,0.001761,1.000000,0.003515,0.8299,8296,1701,0,3


In [45]:
evaluate(df1, "topics")[0].style\
.format(precision=2) \
.set_properties(**{'color': 'red'}, subset=['false negative', 'false positive'])\
.set_properties(**{'color': 'green'}, subset=['true negative', 'true positive'])


,topic,precision,recall,f1_score,accuracy,true negative,false positive,false negative,true positive,auc
0,finance,0.19,0.60,0.29,0.71,65297,24848,3949,5906,0.71
1,apparel and fashion,0.01,0.71,0.01,0.77,77065,22729,60,146,0.83
2,food and dining,0.02,0.86,0.04,0.77,76053,23313,88,546,0.90
3,health and medicine,0.01,0.76,0.01,0.77,77192,22597,51,160,0.84
4,tech and electronics,0.05,0.80,0.09,0.62,60399,37250,473,1878,0.79
5,entertainment,0.01,0.82,0.02,0.64,64048,35504,80,368,0.80
6,home and garden,0.01,0.36,0.01,0.74,74142,25401,291,166,0.56
7,science and environment,0.02,0.64,0.04,0.72,71479,27644,316,561,0.73
8,government and politics,0.04,0.68,0.07,0.56,54017,43481,801,1701,0.65
9,apparel,0.00,0.79,0.00,0.77,76763,23175,13,49,0.89


In [46]:
evaluate(df1, "topics")[1].style\
.format(precision=2) \
.set_properties(**{'color': 'red'}, subset=['false negative', 'false positive'])\
.set_properties(**{'color': 'green'}, subset=['true negative', 'true positive'])


,topic,precision,recall,f1_score,accuracy,true negative,false positive,false negative,true positive
0,finance,0.18,0.65,0.29,0.68,62058,28087,3494,6361
1,apparel and fashion,0.01,0.46,0.02,0.93,92520,7274,112,94
2,food and dining,0.06,0.81,0.10,0.91,90623,8743,118,516
3,health and medicine,0.03,0.82,0.05,0.94,93685,6104,38,173
4,tech and electronics,0.05,0.69,0.09,0.68,66344,31305,732,1619
5,entertainment,0.01,0.93,0.01,0.32,31657,67895,33,415
6,home and garden,0.00,0.49,0.01,0.43,42585,56958,234,223
7,science and environment,0.02,0.57,0.04,0.76,75484,23639,381,496
8,government and politics,0.04,0.76,0.08,0.58,55965,41533,593,1909
9,apparel,0.00,0.92,0.01,0.83,83339,16599,5,57


In [30]:
for body in data['body'][1:10000]:
    if  "forwarded" in body.lower():
        print(body)
        print("--------------------------------------------------")
        print("--------------------------------------------------")
        print("==================================================")
        

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [31]:
data.to_pickle("enron_with_labels")

In [33]:
data1 = pandas.read_pickle("enron_with_labels")

In [37]:
for body in data1['body'][1:1000]:
    if  "------ Forwarded" in body:
        print(body)
        print("--------------------------------------------------")
        print(body.split("----- Forwarded")[0])
        print("==================================================")

---------------------- Forwarded by Judy Hernandez/HOU/ECT on 04/19/2000 
02:01 PM ---------------------------
"SOCORRO HERNANDEZ" <SUHERNAN@LLGM.COM> on 03/29/2000 09:25:37 AM
To: <judy.hernandez@enron.com>, <IRENEpepe@hotmail.com>, "Barbara Monte" 
<BMONTE@LLGM.COM>, "CATHERINE BROWN" <CLBROWN@LLGM.COM>, "CLAUDIA MARTINEZ" 
<CMARTINE@LLGM.COM>, "DAWN SLATER BORN" <DBORN@LLGM.COM>, "DORIS RODRIGUEZ" 
<DRODRIGU@LLGM.COM>, "JESSICA VEGA" <JVEGA@LLGM.COM>, "KAREN MADDREY" 
<KMADDREY@LLGM.COM>, "KATHY MARIE MOORE" <KMOORE@LLGM.COM>, "KATHRYN ANN 
ROBERTS" <KROBERTS@LLGM.COM>, "MARIA SOLIZ" <MSOLIZ@LLGM.COM>, "MARIA 
THOMPSON" <MTHOMPSO@LLGM.COM>, "SANDRA ARRIAGA" <SARRIAGA@LLGM.COM>, "VALERIE 
WALLACE" <VWALLACE@LLGM.COM>, <bburrell@tmh.tmc.edu>
cc:  
Subject: Fwd: A Child's Question

Once upon a time there was a child ready to be born.   The child asked God, 
"They tell me you are sending me to earth tomorrow, but I am so small and 
helpless, how will I be able to live there?"   God repl